In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
%matplotlib inline

In [3]:
!pip install openpyxl 

In [4]:
df = pd.read_csv("../input/zomato-restaurants-data/zomato.csv",encoding="latin-1") 
df.head()

**Infomation about the dataset**

In [5]:
df.columns

In [6]:
df.shape

In [7]:
df.info()

In [8]:
df.describe()

finding missing values in our dataset



In [9]:
df.isnull().sum()

Cuisines have 9 null values

In [10]:
df_country = pd.read_excel('../input/zomato-restaurants-data/Country-Code.xlsx')
df_country

**Merge two tables**

In [11]:
df1 = pd.merge(df,df_country,on='Country Code',how='left')
df1.head()

In [12]:
df1.dtypes

**Exploratory Analysis and Visualization**

List of countrys that present in dataset


In [13]:
print('List of counteris the survey is spread accross')
for x in pd.unique(df1.Country): print(x)
print()
print('Total number to country', len(pd.unique(df1.Country)))

In [14]:
print('Top 3 Country with number of Orders')
df1.Country.value_counts().head(3) 

From this we can say that the maximum number of transactions happen in India

In [15]:
country_names = df1.Country.value_counts().index                 
country_val = df1.Country.value_counts().values  

plt.pie(country_val[:3], labels=country_names[:3],
        autopct='%1.1f%%', pctdistance=0.85,
        )
  
centre_circle = plt.Circle((0, 0), 0.50, fc='white')
fig = plt.gcf()
  
fig.gca().add_artist(centre_circle)
  
plt.title('The top 3 countries with maximum number of orders')

plt.legend(country_names[:3], loc="center")

plt.show()

In [16]:
import plotly.graph_objects as go
country_names = df1.Country.value_counts().index                 
country_val = df1.Country.value_counts().values  

# pull is given as a fraction of the pie radius
fig = go.Figure(data=[go.Pie(labels=country_names[:3], values=country_val[:3], pull=[0.5, 0, 0], 
                             title='Population of European continent')])
fig.show()

#Pulling sectors out from the center
# https://plotly.com/python/pie-charts/

**Exploring raiting**

In [17]:
df2 = df1.groupby(['Aggregate rating','Rating color', 'Rating text']).size().reset_index().rename(columns={0:'Rating Count'})
df2.head()

The above information helps us to understand the relation between Aggregate rating, color, and text. 
The following color assigned to the ratings:

* Rating 0 — White — Not rated
* Rating 1.8 to 2.4 — Red — Poor
* Rating 2.5 to 3.4 — Orange — Average
* Rating 3.5 to 3.9 — Yellow — Good
* Rating 4.0 to 4.4 — Green — Very Good
* Rating 4.5 to 4.9 — Dark Green — Excellent

**Data Visulizing**

User Raiting

In [18]:
plt.rcParams['figure.figsize'] = (12,6)
sns.barplot(x='Aggregate rating',y='Rating Count',hue = 'Rating color',data=df2,
            palette=['blue','red','orange','yellow','green','green'])
plt.title('User Ratings')

**Maximum restaurants seems to have gone No ratings. Let us check if these restaurants belong to some specific country.**

In [19]:
No_rating = df1[df1['Rating color']=='White'].groupby('Country').size().reset_index().rename(columns={0:'Rating Count'})
No_rating

India seems to have maximum unrated restaurants. 

**Currency Usage By Each Country**

In [20]:
country_currency = df1[['Country','Currency']].groupby(['Country','Currency']).size().reset_index(name='count').drop('count', axis=1, inplace=False)
country_currency.sort_values('Currency').reset_index(drop=True)

**Distribution of Online Delivery**

In [21]:
plt.figure(figsize=(12,6))
plt.title(' Distribution of Online Delivery')
plt.pie(df1['Has Online delivery'].value_counts(), 
        labels=df1['Has Online delivery'].value_counts().index,
        autopct='%1.1f%%', startangle=180);

Only 25% of restaurants accept online delivery. This data might be biased as we have the maximum number of restaurants listed here are from India. 

In [22]:
import plotly.graph_objects as go

labels = list(df1.City.value_counts().head(20).index)
values = list(df1.City.value_counts().head(20).values)

# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values,hole=0.6,title="Zomato's Presence Citywise")])
fig.show()


#Donut chart
#https://plotly.com/python/pie-charts/

The data seems to be skewed towards New Delhi, Gurgaon, and Noida. I see minimal data for other cities. 

**Q & A**

*From which Locality maximum hotels are listed in Delhi Zomato*

In [23]:
import plotly.graph_objects as go
Delhi = df1[(df1.City == 'New Delhi')]
fig = go.Figure(data=[go.Bar(
    x=Delhi.Locality.value_counts().head(10).index,
    y=Delhi.Locality.value_counts().head(10),
    
)])

fig.show()

Connaught place seems to have a high no of restaurants registered with Zomato.

*What kind of Cuisine do these highly-rated restaurants offer*

top 10 cuisine availabel

In [24]:
cuisine_val = df1.Cuisines.value_counts()             
cuisine_label = df1.Cuisines.value_counts().index 
fig = go.Figure(data=[go.Pie(labels=cuisine_label[:10], values=cuisine_val[:10], textinfo='label+percent',
                             insidetextorientation='radial',title='Top 10 Cuisine'
                            )])
fig.show()

*How many of such restaurants accept online delivery*

In [25]:
top_locality = Delhi.Locality.value_counts().head(10)
plt.figure(figsize=(12,6))
ax = sns.countplot(y= "Locality", hue="Has Online delivery", data=Delhi[Delhi.Locality.isin(top_locality.index)])
plt.title('Resturants Online Delivery');

Online Delivery seems to be on the higher side in Defence colony and Malviya Nagar.
Other than Shahdara locality ,in other restaurants localities accept online delivery.

*Rating VS Cost of dinning*

In [26]:
import plotly.express as px

fig = px.scatter(Delhi, x="Average Cost for two", y="Aggregate rating", color='Price range',)
fig.show()

I observe there is no linear relation between price and rating.


*Location of Highly rated restaurants across New Delhi*

In [27]:
Delhi['Rating text'].value_counts()

In [28]:
High_rating = Delhi[Delhi['Rating text'].isin(['Excellent'])]
fig = px.scatter_mapbox(High_rating, lat="Latitude", lon="Longitude", hover_name="City",
                        hover_data=["Aggregate rating", "Restaurant Name"],
                        color_discrete_sequence=["fuchsia"], zoom=10, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(title='Highle rated Resturants Location',
                  autosize=True,
                  hovermode='closest',
                  showlegend=False)
fig.update_layout(
    autosize=False,
    width=800,
    height=500,)

fig.show()